In [163]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [164]:
from datamanager import datamanager as mg
mg = mg()

In [166]:
def link_list(multi_layered):  
    linked = []    
    for i in range(len(multi_layered)):
        linked += multi_layered[i]
    return linked 

In [165]:
def get_adj_naver_list(num_code, count):
    # num_code = '005930'
    # count = 500 -> 실제 거래일 수와 달라서 to_date가 0이 아닐 수 있다. 그리고 시작 시점을 조절하지 못함.
    url = 'https://fchart.stock.naver.com/sise.nhn'
    payload = {
        'symbol' : num_code,
        'timeframe' : 'day',
        'count' : count,
        'requestType' : 0
    }
    req = requests.get(url, params=payload).text
    soup = bs(req,'lxml')
    row_list = []
    for item in soup.findAll('item'):
        row = item['data'].split('|')
        row.append('A'+ num_code)
        row_list.append(row)
    
    return row_list

In [167]:
def get_naver_companies():
    """
    naver는 
    """
    url = 'https://finance.naver.com/sise/sise_market_sum.nhn'
    payload = dict()
    company_pages = []
    for j in [0,1]: # 0은 코스피, 1은 코스닥
        for i in range(1, 33):
            payload['sosok'] = j
            payload['page'] = i
            req = requests.get(url, params=payload).text
            soup = bs(req, 'lxml')    
            company_pages.append(pd.read_html(req)[1])

    df = pd.concat(company_pages)
    companies = df['종목명']
    companies.dropna(inplace=True)
    companies.reset_index(drop=True, inplace=True)
    return companies

In [191]:
x = pd.read_pickle('../Database/10year.pickle')

In [194]:
test = x[x['short_code'] == 'A149940'].copy()

In [195]:
test

,open,high,low,close,volume,short_code
date,,,,,,
2012-09-25,4450,5000,3785,3785,2263306,A149940
2012-09-26,3770,4350,3765,4350,1021390,A149940
2012-09-27,4250,4950,4175,4460,2573751,A149940
2012-09-28,4600,4795,4075,4140,892922,A149940
2012-10-02,4185,4185,3835,3915,505847,A149940
...,...,...,...,...,...,...
2020-02-07,0,0,0,155,0,A149940
2020-02-10,0,0,0,155,0,A149940
2020-02-11,0,0,0,155,0,A149940


In [190]:
companies

,codeName,short_code
0,삼성전자,A005930
1,SK하이닉스,A000660
2,삼성전자우,A005935
3,삼성바이오로직스,A207940
4,NAVER,A035420
...,...,...
2313,이베스트이안스팩1호,A323210
2314,IBKS제12호스팩,A335870
2315,대호피앤씨우,A021045
2316,소프트센우,A032685


In [94]:
def get_naver_total(days):
    com_list = mg.full_code['codeName']
    code_list = mg.full_code['short_code']

    com_code = dict()
    for i, company in enumerate(com_list):
        code = code_list[i]
        com_code[company] = code

    super_list = []
    for i, com in enumerate(companies):
        try:
            short_code = com_code[com]
            num_code = short_code[1:]
            super_list.append(get_adj_naver_list(num_code, count=2500))
        except KeyError:
            pass


    linked = pd.DataFrame(link_list(super_list)) 
    linked.columns = ['date', 'open' ,'high', 'low', 'close', 'volume','short_code']
    linked['date'] = linked['date'].apply(lambda x : pd.to_datetime(x))   
    linked[['open', 'high', 'low', 'close', 'volume']] = linked[['open' ,'high', 'low', 'close', 'volume']].applymap(lambda x : int(x))
    linked.set_index('date', inplace=True)
    
    return linked

    linked.to_pickle('../Database/10year.pickle')

In [95]:
linked.info()

In [10]:
saved = pd.read_pickle('../Database/10year.pickle')

In [84]:
saved

,open,high,low,close,volume,short_code
date,,,,,,
2009-12-18,15200,15580,15159,15460,406524,A005930
2009-12-21,15460,15540,15339,15420,291763,A005930
2009-12-22,15500,15620,15460,15620,245910,A005930
2009-12-23,15760,15760,15560,15680,250159,A005930
2009-12-24,15700,15880,15640,15800,407107,A005930
...,...,...,...,...,...,...
2020-02-04,401,515,270,316,29193142,A122800
2020-02-05,342,342,213,229,17756257,A122800
2020-02-06,180,189,132,132,24389271,A122800


In [15]:
saved.iloc[-1].name

Timestamp('2020-02-10 00:00:00')

In [16]:
from datetime import datetime

In [17]:
today = datetime.today()

In [18]:
saved.iloc[-1].name < today

True

In [134]:
def get_current_price():
    url = 'https://finance.naver.com/sise/field_submit.nhn?menu=market_sum&returnUrl=http%3A%2F%2Ffinance.naver.com%2Fsise%2Fsise_market_sum.nhn%3F%26page%3D1&fieldIds=quant&fieldIds=open_val&fieldIds=high_val&fieldIds=low_val'
    payload = dict()
    company_pages = []
    for sosok in [0,1]: # 0은 코스피, 1은 코스닥
        payload['sosok'] = sosok
        for page_num in range(1, 33): # 마지막 페이지는 거의 33
            print('페이지 수 : {}'.format(page_num))
            payload['page'] = page_num
            req = requests.get(url, params=payload).text
            company_pages.append(pd.read_html(req)[1][['시가', '고가', '저가', '현재가', '거래량', '종목명']])
            if page_num==1: break
        if sosok==0: break

    df = pd.concat(company_pages)
    companies = df
    companies.dropna(inplace=True)
    companies.reset_index(drop=True, inplace=True)
    companies.index = [datetime.today()]*len(companies)
    companies.columns = ['open', 'high', 'low', 'close', 'volume', 'codeName']
    
    return companies

In [135]:
test = get_current_price()

페이지 수 : 1


In [136]:
full_code = mg.full_code
merged = test.merge(full_code, on=['codeName'])
result = merged[['open', 'high', 'low', 'close', 'volume', 'short_code']].copy()
result.index= [pd.to_datetime(datetime.today().strftime('%Y%m%d'))] * len(result)
concated = pd.concat([saved, result])

In [140]:
test.head(3)

,open,high,low,close,volume,codeName
2020-02-13 12:58:15.130551,61200.0,61600.0,61000.0,61200.0,9790422.0,삼성전자
2020-02-13 12:58:15.130551,101000.0,103500.0,100500.0,102000.0,1848436.0,SK하이닉스
2020-02-13 12:58:15.130551,51400.0,51900.0,51400.0,51500.0,911972.0,삼성전자우


In [141]:
merged = test.merge(full_code, on=['codeName'])

In [144]:
merged[['open', 'high', 'low', 'close', 'volume', 'short_code']].head(3)

,open,high,low,close,volume,short_code
0,61200.0,61600.0,61000.0,61200.0,9790422.0,A005930
1,101000.0,103500.0,100500.0,102000.0,1848436.0,A000660
2,51400.0,51900.0,51400.0,51500.0,911972.0,A005935


In [152]:
result = merged[['open', 'high', 'low', 'close', 'volume', 'short_code']].copy()
result.index= [pd.to_datetime(datetime.today().strftime('%Y%m%d'))] * len(result)
result.head(3)

,open,high,low,close,volume,short_code
2020-02-13,61200.0,61600.0,61000.0,61200.0,9790422.0,A005930
2020-02-13,101000.0,103500.0,100500.0,102000.0,1848436.0,A000660
2020-02-13,51400.0,51900.0,51400.0,51500.0,911972.0,A005935


In [153]:
saved.head(3)

,open,high,low,close,volume,short_code
date,,,,,,
2009-12-18,15200,15580,15159,15460,406524,A005930
2009-12-21,15460,15540,15339,15420,291763,A005930
2009-12-22,15500,15620,15460,15620,245910,A005930


In [156]:
concated = pd.concat([saved, result])

In [162]:
saved.index.unique().sort_values()[-1]

Timestamp('2020-02-10 00:00:00')